**Step 3: Build a Widget that scans through polaritonic potential energy surfaces as a function of coupling strength $ g$**

We will build a widget that will provide you with an interactive plot of the polaritonic potential energy surfaces as a function of $g$.  This will help us to vizualize the way that the fundamental coupling strength impacts the 
shapes of the polaritonic potential surface, and can also help us to estimate a quantitative relationship
between the coupling strength $g = {\bf A} \cdot \mu$ and the so-called Rabi splitting that occurs between 
the surfaces at a particular value of the bondlength $R_{deg}$, where we define $R_{deg}$ as the bondlength 
at which the following is satisfied for a particular value of the photon energy $\omega_0$:

$$ E_g(R_{deg}) + \hbar \omega_0 = E_e(R_{deg}) \tag{5}$$.

If we set $\omega_0$ to be equal to the transition energy at $R = 2.5$ Angstroms as before, then we have 
$R_{deg} = 2.5$ Angstroms by definition. As we worked out in an earlier cell, this value of $\omega_0$ would
correspond to approximately $0.1487$ atomic units or $4.048$ eV.  We will use this value for $\omega$ for the rest of this notebook.

We already have a function that will build the Rabi Hamiltonian as a function of the fundamental coupling strength, the photon energy, and the bondlength value.  The next step is to build a function that can be called by the widget to return the lower- and upper-poloriton potential energy surfaces ($E_{LP}(R)$ and $E_{UP}(R)$) by diagonalizing this Hamiltonian and returning the 2nd and 3rd eigenvalues, respectively.  We will use the `eigh` function of numpy's linear algebra package (given the alias `LA` in our import statement at the top of the notebook) to diagonalize the Hamiltonian and store the eigenvalues.  The syntax follows:

`vals, vecs = LA.eigh(matrix)`

where `vals` are the eigenvalues, `vecs` are the eigenvectors, and `matrix` is the Hermitian matrix.

The following function `polariton_surfaces` will compute and return $E_{LP}(R)$ and $E_{UP}(R)$ in this way.

In [ ]:
%matplotlib widget
import ipympl
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
from scipy.interpolate import InterpolatedUnivariateSpline
from numpy import linalg as LA


In [ ]:
# Create a slider widget and a widget to hold the plot.
slider = widgets.FloatSlider(description = r'$g$',
                            value = 0,
                            min = 0,
                            max = 5,  
                            step = 1)
# We can use HTML styles to control the appearance of the widget.
plot_widget = widgets.Output(layout = {'width':'100%', 'border': '1px solid black'})

In [ ]:
# function to return upper-polariton surface
def lp(amplitude, omega, r_val, g_spline, e_spline, tdm_spline):
    H = np.zeros((3,3))
    lp_vals = np.zeros_like(r_val)
    for i in range(0, len(r_val)):
        H[0,0] = g_spline(r_val[i])
        H[1,1] = g_spline(r_val[i]) + omega
        H[1,2] = amplitude * tdm_spline(r_val[i])
        H[2,1] = amplitude * tdm_spline(r_val[i])
        H[2,2] = e_spline(r_val[i])
        vals, vecs = LA.eigh(H)
        lp_vals[i] = vals[1]
        
    return lp_vals


# function to return lower-polariton surface
def up(amplitude, omega, r_val, g_spline, e_spline, tdm_spline):
    H = np.zeros((3,3))
    up_vals = np.zeros_like(r_val)
    for i in range(0, len(r_val)):
        H[0,0] = g_spline(r_val[i])
        H[1,1] = g_spline(r_val[i]) + omega
        H[1,2] = amplitude * tdm_spline(r_val[i])
        H[2,1] = amplitude * tdm_spline(r_val[i])
        H[2,2] = e_spline(r_val[i])
        vals, vecs = LA.eigh(H)
        up_vals[i] = vals[2]
        
    return up_vals

# bondlength grid
rs = np.array([1.3, 1.42, 1.54, 
               1.6600000000000001, 1.78, 1.9, 
               2.02, 2.14, 2.26, 2.38, 2.5, 
               2.62, 2.74, 2.8600000000000003, 
               2.98, 3.0999999999999996, 3.2199999999999998, 
               3.34, 3.46, 3.58])

# ground-state energy array from TDDFT
Eg = np.array([-200.34317863, -200.36733437, -200.3788335,  -200.38211913, -200.38005015,
 -200.37454644, -200.36693436, -200.35813969, -200.34880828, -200.33938762,
 -200.33018284, -200.32139619, -200.3131552,  -200.30553287, -200.29856231,
 -200.29224756, -200.28657217, -200.2815057,  -200.27700922, -200.27303916])

# excited-state energy array from TDDFT
Ee = np.array([-200.11946659, -200.15343212, -200.17455494, -200.18699014, -200.19340378,
 -200.19564856, -200.19506567, -200.19266834, -200.18922925, -200.18533058,
 -200.18141088, -200.17778616, -200.17467719, -200.17223017, -200.17053044,
 -200.16961006, -200.16945866, -200.17002824, -200.17124271, -200.17300921])

# transition dipole moment array from TDDFT
mu = np.array([1.41026992, 1.46500868, 1.53226812, 1.60873137, 1.69113032, 1.77658259,
 1.86280274, 1.9479899,  2.03067753, 2.10965627, 2.18375603, 2.25178818,
 2.31244456, 2.36432877, 2.40606639, 2.43656128, 2.45526201, 2.46232396,
 2.45859393, 2.44546058])

# fit splines
Eg_spline = InterpolatedUnivariateSpline(rs, Eg, k=3)
Ee_spline = InterpolatedUnivariateSpline(rs, Ee, k=3)
mu_spline = InterpolatedUnivariateSpline(rs, mu, k=3)

# photon frequency
om = 4.048 / 27.211


In [ ]:

# Turn off interactive mode before creating the 
# plot so it doesn't display too early.
plt.ioff() # Turn off interactive mode
fig, ax = plt.subplots(constrained_layout = True, figsize=[5,4]);
plt.ion() # Turn on interactive mode
line1, = ax.plot(rs, lp(0.00, om, rs, Eg_spline, Ee_spline, mu_spline));
line2, = ax.plot(rs, up(0.00, om, rs, Eg_spline, Ee_spline, mu_spline));
ax.set_ylabel(r'$E(R)$')
ax.set_xlabel(r'$R$')
ax.grid(True)

# force the figure to display in the plot_widget
with plot_widget:
    display(fig.canvas)


In [ ]:
# Create a function to re-draw the plot using the value of the slider. It is 
# faster to change the y-data but you can erase the plot and draw a new one.
def update(value):
    '''We can use `slider.value` or `value.new` to get the new slider value.'''
    with plot_widget:
        line1.set_ydata(lp(slider.value*0.001, om, rs, Eg_spline, Ee_spline, mu_spline))
        line2.set_ydata(up(slider.value*0.001, om, rs, Eg_spline, Ee_spline, mu_spline))
        fig.canvas.draw()

# Set an observer to call `update` whenever the value changes.
slider.observe(update)

Use the slider below to change the coupling strength, $g$.

In [ ]:
widgets.VBox([plot_widget, slider])

**Question 6**  We can define the Rabi splitting as the difference between the upper- and lower-polariton surfaces
right at $R = R_{deg}$:

$$ E_{RS} = E_{UP}(R_{deg}) - E_{LP}(R_{deg})\tag{6}. $$

Use the slider to estimate the different values of $E_{RS}$ as a function of $g$ and fill in 
the value in the table below.  *Note:* The $A$ value displayed by the widget is scaled by $10^{-3}$ atomic units before computing the potential energy surfaces. 

| $g$  | $E_{UP}$ |$E_{LP}$ | $E_{RS}$ |
| :- | :- |:- |:-: |
| 0 | -200.1810 |-200.1810 |0|
| 1 | -200.1785 |-200.1832|0.0046 |
| 2 | - |- |- |
| 3 | - |-|- |
| 4 | - |-| - |
| 5 |- |-| - |

    
**Question 7** Create a plot of $E_{RS}$ vs $g$.  Do you think the relationship is linear, quadratic, cubic, or a higher-order polynomial?  Justify your answer in a few sentences.